In [43]:
import requests
from bs4 import BeautifulSoup
from collections import deque
import pandas as pd
import time
import logging

from threading import Thread
from queue import Queue

In [44]:
LIMIT = 20
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [45]:
df = pd.read_csv('dataset_filter_podla_reachability.csv')
df.head()

,Unnamed: 0,name,link,type
0,0,Australian National University,http://www.anu.edu.au/,Education
1,1,Monash University,http://www.monash.edu/,Education
2,2,University of Queensland,http://www.uq.edu.au/,Education
3,3,Macquarie University,http://mq.edu.au/,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,Education


In [46]:
url_adresy = [[i, link] for i, link in enumerate(df['link'])]
url_adresy

[[0, 'http://www.anu.edu.au/'],
 [1, 'http://www.monash.edu/'],
 [2, 'http://www.uq.edu.au/'],
 [3, 'http://mq.edu.au/'],
 [4, 'https://www.unsw.edu.au/'],
 [5, 'http://www.ncl.ac.uk/'],
 [6, 'https://www.uow.edu.au/index.html'],
 [7, 'http://www.unimelb.edu.au/'],
 [8, 'http://www.utas.edu.au/'],
 [9, 'http://www.adelaide.edu.au/'],
 [10, 'http://www.jcu.edu.au/'],
 [11, 'http://www.flinders.edu.au/'],
 [12, 'https://www.rmit.edu.au/'],
 [13, 'http://www.latrobe.edu.au/'],
 [14, 'http://www.vu.edu.au/'],
 [15, 'http://www.une.edu.au/'],
 [16, 'http://www.deakin.edu.au/'],
 [17, 'http://www.griffith.edu.au/'],
 [18, 'https://www.cqu.edu.au/'],
 [19, 'https://www.unisa.edu.au/'],
 [20, 'http://www.swinburne.edu.au/'],
 [21, 'http://bond.edu.au/'],
 [22, 'http://www.usc.edu.au/'],
 [23, 'http://www.csu.edu.au/'],
 [24, 'http://www.canberra.edu.au/'],
 [25, 'https://federation.edu.au/'],
 [26, 'http://www.wehi.edu.au/'],
 [27, 'http://www.cdu.edu.au/'],
 [28, 'http://www.aims.gov.au/'],
 

In [47]:
def getdata(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=5)
        return r.text
    except:
        return None
    # r = requests.get(url, headers=HEADERS, timeout=5)
    # return r.text

def get_links(website_link):
    newly_found_links = deque()
    html_data = getdata(website_link)
    if html_data is None:
        return []

    soup = BeautifulSoup(html_data, "html.parser")
    for link in soup.find_all("a", href=True)[:LIMIT]:
        # print(link)

        # Append to list if new link contains original link
        if str(link["href"]).startswith((str(website_link))):
            newly_found_links.append(link["href"])

        # Include all href that do not start with website link but with "/"
        if str(link["href"]).startswith("/"):
            link_with_www = website_link + link["href"][1:]
            newly_found_links.append(link_with_www)

        # include subdomains i.e domain: upjs.sk, subdomain: science.upjs.sk
        if website_link.split("//")[1].split("/")[0].replace("www.", "") in str(link["href"]):
            newly_found_links.append(str(link["href"]))
    return newly_found_links

def get_subpages_of_website(website_url, i):
    valid_pages = 0
    links_to_be_checked = deque([website_url])
    already_checked_links = deque()
    file_name = f"../tmp//{i}.txt"

    while links_to_be_checked:
        link = links_to_be_checked.popleft()
        if link not in already_checked_links:
            links_to_be_checked.extend(get_links(link))
            with open(file_name, "a") as file:
                file.write(link+"\n")
                valid_pages += 1

            # Print some statements
            total = len(links_to_be_checked) + len(already_checked_links)
            print(f"Total n.o. links: {total}")
            print(f"Checked: {len(already_checked_links)} / {total}\n")
            print(f"Written: = {valid_pages}")

        already_checked_links.append(link)

        # print(len(links_to_be_checked))
        # print(links_to_be_checked)

        if valid_pages >= LIMIT:
            print("Exited forcefully")
            break
    else:
        print("Exited peacefully")

    # file = open(file_name, "a")
    # for item in already_checked_links:
    #     file.write(item+"\n")
    # file.close()

    return True



def crawl(q, results):
    while not q.empty():
        work = q.get()
        i = work[0] # index
        adresa = work[1] # URL adresa
        print(adresa)

        try:
            page = requests.get(adresa, headers=HEADERS, timeout=5)
            if page.status_code == 200:
                # logging.info(f'{i}: {adresa}: is reachable')
                print(f'{i}: {adresa}: is reachable')
                results[i] = True

                # soup = BeautifulSoup(page.content, "html.parser")
                get_subpages_of_website(adresa, i)

            else:
                # logging.info(f'{i}: {adresa}: is Not reachable, status_code: {my_page.status_code}')
                print(f'{i}: {adresa}: is Not reachable, status_code: {page.status_code}')
                # results[i] = False
        except requests.exceptions.RequestException as e:
            # logging.info(f'{i}: {adresa}: is Not reachable')
            print(f'{i}: {adresa}: is Not reachable')

        results[i] = True

        q.task_done()
    return True

In [48]:
# Vlozenie jednotlivych taskov (stranok) do radu (Queue)
threads = []
q = Queue(maxsize=0)
# n_jobs = min(1000000000, len(url_adresy))
from_n_jobs, n_jobs = 149, 150


# results = [{} for x in url_adresy]
results = [False for x in url_adresy]
for i, url in url_adresy[from_n_jobs: n_jobs]:
    q.put((i, url))


# Vykonanie taskov pomocou multithreading-u
cas = time.time()

#Starting worker threads on queue processing
for i in range(n_jobs):
    logging.debug('Starting thread ', i)
    worker = Thread(target=crawl, args=(q, results))
    # worker.setDaemon(True)    #setting threads as "daemon" allows main program to exit eventually even if these dont finish correctly.
    worker.daemon = True
    worker.start()


#now we wait until the queue has been processed
q.join()

# logging.info('All tasks completed.')
print('All tasks completed.')

print(f"Websites checked: {n_jobs}")
print(f"Finished in {(time.time() - cas):.1f} seconds.")

https://sgm.sabanciuniv.edu/
149: https://sgm.sabanciuniv.edu/: is reachable
Total n.o. links: 18
Checked: 0 / 18

Written: = 1
Total n.o. links: 29
Checked: 2 / 29

Written: = 2
Total n.o. links: 48
Checked: 3 / 48

Written: = 3
Total n.o. links: 69
Checked: 4 / 69

Written: = 4
Total n.o. links: 90
Checked: 5 / 90

Written: = 5
Total n.o. links: 111
Checked: 6 / 111

Written: = 6
Total n.o. links: 131
Checked: 7 / 131

Written: = 7
Total n.o. links: 152
Checked: 8 / 152

Written: = 8
Total n.o. links: 172
Checked: 9 / 172

Written: = 9
Total n.o. links: 192
Checked: 10 / 192

Written: = 10
Total n.o. links: 203
Checked: 11 / 203

Written: = 11
Total n.o. links: 226
Checked: 12 / 226

Written: = 12
Total n.o. links: 241
Checked: 14 / 241

Written: = 13
Total n.o. links: 256
Checked: 16 / 256

Written: = 14
Total n.o. links: 276
Checked: 18 / 276

Written: = 15
Total n.o. links: 297
Checked: 20 / 297

Written: = 16
Total n.o. links: 318
Checked: 21 / 318

Written: = 17
Total n.o. links